## SVM For Weather Satellite

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split  # type: ignore
from sklearn.preprocessing import StandardScaler  # type: ignore

# import dependencies
import pandas as pd
import sqlalchemy as sq
import sys, os
from imblearn.combine import SMOTEENN
from sklearn.svm import SVC  # type: ignore

from sklearn.metrics import (  # type: ignore
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
)

sys.path.append("../../")
os.chdir("../../")
from ModelBuilderMethods import getConn, extractYears

In [2]:
# unlimited line output
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 500)

### <u>**Step 1**</u>: Data Selection

In this step, we would choose the particular data/table, pick attributes from existing tables. Further aggregation/feature engineer can be done here to support the point of the research.

Particular, for this notebook, we grab the following data and merge them (on year, district) into a single table:
- Monthly weather satellite data
- ergot data (downgrade)

In [3]:
# Set the query text
weatherSatQuery = sq.text(
    """
    SELECT * from dataset_cross_monthly_sat
"""
)

ergotTargetQuery = sq.text(
    """
    SELECT year, district, downgrade from ergot_sample_feat_eng
"""
)

In [4]:
conn = getConn("./.env")

satelliteDf = pd.read_sql(weatherSatQuery, conn)
ergotTargetDf = pd.read_sql(ergotTargetQuery, conn)

conn.close()
del conn

In [5]:
tempdf = satelliteDf

# merge on year and district
datasetDf = pd.merge(ergotTargetDf, tempdf, on=["year", "district"], how="left")
del ergotTargetDf
del tempdf

In [6]:
# encode district
datasetDf["district"] = datasetDf["district"].astype("category")

temp = pd.get_dummies(datasetDf["district"], prefix="district", drop_first=True)
datasetDf = pd.concat([datasetDf, temp], axis=1)

datasetDf = datasetDf.drop(columns=["district"])

del temp

### <u>**Step 2**</u>: Splitting dataset

- We split the whole dataset into the train/test split. Particularly, split them by year (1995 - 2015 for training, 2016 - 2020 for testing) since this is a time series data.

In [7]:
# train 1995 - 2015 test 2016 - 2020
trainDf = extractYears(datasetDf, 1995, 2015)
testDf = extractYears(datasetDf, 2016, 2020)
del datasetDf

In [8]:
# drop year
trainDf = trainDf.drop(columns=["year"])
testDf = testDf.drop(columns=["year"])

In [9]:
# pre balancing check
# print value counts downgrade
print(trainDf["downgrade"].value_counts())
print(testDf["downgrade"].value_counts())

downgrade
False    122202
True       2082
Name: count, dtype: int64
downgrade
False    26307
True      1016
Name: count, dtype: int64


In [10]:
# count nan
print(trainDf.isna().sum())
# set nan to 0
trainDf = trainDf.fillna(0)

downgrade                           0
1:min_dewpoint_temperature          0
1:min_temperature                   0
1:min_evaporation_from_bare_soil    0
1:min_skin_reservoir_content        0
                                   ..
district_4830                       0
district_4840                       0
district_4850                       0
district_4860                       0
district_4870                       0
Length: 687, dtype: int64


### <u>**Step 3**</u>: [Balancing the dataset](https://imbalanced-learn.org/stable/)

- Our dataset is unbalanced and can lead to bias when training/testing. Balacing step would help to eliminate the bias of the dataset, thus provide more reliable results.

In [11]:
balancer = SMOTEENN(sampling_strategy=1, random_state=42)
balancedTrainDfX, balancedTrainDfY = balancer.fit_resample(
    trainDf.drop(columns="downgrade"), trainDf["downgrade"]
)

In [12]:
print(balancedTrainDfY.value_counts())

downgrade
False    115239
True      25156
Name: count, dtype: int64


### <u>**Step 4**</u>: Regularization / Normalization
some blurb about scalers  

1. [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)             
2. [MaxAbsScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html)  
3. [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)  
4. [RobustScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)  
5. [Normalizer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html)  
6. [PowerTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html)  
7. [QuantileTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.QuantileTransformer.html)  

In [13]:
def printMetrics(model_name, y_true, y_pred):
    print(model_name)
    print("Accuracy: ", accuracy_score(y_true, y_pred))
    print("Precision: ", precision_score(y_true, y_pred))
    print("Recall: ", recall_score(y_true, y_pred))
    print("F1: ", f1_score(y_true, y_pred))
    print("ROC AUC: ", roc_auc_score(y_true, y_pred))
    print("Classification Report: \n", classification_report(y_true, y_pred))
    print()

### <u>**Step 5**</u>: SVM Model

##### <u>**Step 5.1**</u>: Initialize the model

In [14]:
svm_model = SVC(kernel="rbf", C=1)
balanced_svm_model = SVC(kernel="rbf", C=1)

##### <u>**Step 5.2**</u>: Fit the training data to the model

In [16]:
svm_model.fit(trainDf.drop(columns="downgrade"), trainDf["downgrade"])
balanced_svm_model.fit(balancedTrainDfX, balancedTrainDfY)

##### <u>**Step 5.3**</u>: Test the model on the testing dataset

In [ ]:
predictions_svm = svm_model.predict(testDf.drop(columns="downgrade"))
predictions_balanced_svm = balanced_svm_model.predict(testDf.drop(columns="downgrade"))

In [ ]:
print(pd.DataFrame(predictions_svm).value_counts())
print(pd.DataFrame(predictions_balanced_svm).value_counts())

##### <u>**Step 5.4**</u>: Evaluate models based on different metrics:
- ACCURACY:
- PRECISION:
- RECALL:
- F1:
- ROC AUC:

In [ ]:
printMetrics("SVM imbalanced train set", testDf["downgrade"], predictions_svm)
printMetrics(
    "SVM balanced train set",
    testDf["downgrade"],
    predictions_balanced_svm,
)

##### <u>**Step 5.5**</u>: Hypertune the hyperparameters on different kernels, C

In [ ]:
kernals = ["linear", "poly", "rbf", "sigmoid"]
c_list = [0.1, 1, 10, 100]

for k in kernals:
    for c in c_list:
        print("--------------------------------------")
        print(f"Kernal = {k}, C = {c}")

        # Train & Test model on unbalanced data
        unbal_model = SVC(kernel=k, C=c)
        unbal_model.fit(trainDf.drop(columns="downgrade"), trainDf["downgrade"])
        y_pred = unbal_model.predict(testDf.drop(columns="downgrade"))
        printMetrics("SVM imbalanced train set", testDf["downgrade"], unbal_model)

        # Train & Test model on balanced data
        bal_model = SVC(kernel=k, C=c)
        bal_model.fit(trainDf.drop(columns="downgrade"), trainDf["downgrade"])
        y_pred = bal_model.predict(testDf.drop(columns="downgrade"))
        printMetrics(
            "SVM balanced train set",
            testDf["downgrade"],
            bal_model,
        )